In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import os
import numpy as np
from keras.callbacks import LambdaCallback
import sys
import random
import string
import re

Using TensorFlow backend.


### Preprocessing 

In [2]:
# read in text, lower case
text_full = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read().lower()

In [3]:
# remove numbers
text_no_num = ''.join([i for i in text_full if not i.isdigit()])
# remove new lines
text_no_nline = text_no_num.replace("\n", '')

In [4]:
# remove punctuation and tabs

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
no_punct = ""
for char in text_no_nline:
    if char not in punctuations:
        no_punct = no_punct + char

text = re.sub(' +', ' ', no_punct)

In [5]:
# from https://keras.io/examples/lstm_text_generation/

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 27


In [6]:
chars

[' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [7]:
# from https://keras.io/examples/lstm_text_generation/
# cut the text in semi-redundant sequences of maxlen characters

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 29602


### turn to one hot encoding 

In [8]:
# from https://keras.io/examples/lstm_text_generation/

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

### define model

In [12]:
# define model
model = Sequential()
model.add(LSTM(125, input_shape=(x.shape[1], x.shape[2])))
model.add(Dense(len(chars), activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 125)               76500     
_________________________________________________________________
dense_1 (Dense)              (None, 27)                3402      
Total params: 79,902
Trainable params: 79,902
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [172]:
# temperature = 1.0

In [173]:
# # from https://keras.io/examples/lstm_text_generation/

# def sample(preds, temperature=1.0):
#     # helper function to sample an index from a probability array
    
#     preds = np.asarray(preds).astype('float64')
#     # take log of predictions and scale by temperature (why take log?)
#     preds = np.exp(np.log(preds) / temperature)
#     # scale back to 1 
#     preds = preds / np.sum(preds)
#     # sample a distribution, return an array with 1 at choice
#     probas = np.random.multinomial(1, preds, 1)
#     # return argument (index) that produced choice
#     return np.argmax(probas)

In [485]:
# # from https://keras.io/examples/lstm_text_generation/
# # change to prevent log(0)

# def sample(preds, temperature=1.0):
#     # helper function to sample an index from a probability array
    
#     preds = np.asarray(preds).astype('float64')
    
#     if any(preds==0) or any(np.log(preds) < -15):
# #         preds[np.where(preds==0)[0][0]] = np.nan
#         preds[np.where(np.log(preds) < -15)[0][0]] = np.nan
              
#     else:
#         pass
        
#     # take log of predictions and scale by temperature (why take log?)
#     preds = np.exp(np.log(preds) / temperature)
#     # scale back to 1 
#     preds = preds / np.nansum(preds)
    
    
#     # convert nans back to 0
#     if any(np.isnan(preds)):
#         preds[np.where(np.isnan(preds))[0][0]] = 0.
#     else:
#         pass
    
#     # sample a distribution, return an array with 1 at choice
#     probas = np.random.multinomial(1, preds, 1)
#     # return argument (index) that produced choice
#     return np.argmax(probas)

In [15]:
# modified from https://keras.io/examples/lstm_text_generation/

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    
    preds = np.asarray(preds).astype('float64')
    # scale by temp
    preds = preds / temperature

    # rescale to 1
    preds = preds/ np.sum(preds)
    # sample a distribution, return an array with 1 at choice
    probas = np.random.multinomial(1, preds, 1)
    # return argument (index) that produced choice
    return np.argmax(probas)

In [17]:
model.fit(x, y, batch_size=32, epochs=10, verbose=2, callbacks=[print_callback])

Epoch 1/10
 - 28s - loss: 2.5475 - accuracy: 0.2645

----- Generating text after Epoch: 0
----- temperature: 0.25
----- Generating with seed: "saucy bark inferior far to hison your br"
saucy bark inferior far to hison your brvciss aps aptstty than haml twes aaig wout akit ot wist fhesuaceetilidaad thiu favingis tftus amd men art amd thild croty ceapasin hit wish to te may tltem mriin ti the ave byt dinghald aims hestive wesoeskrt aotthetuthoui oar fyceansy sas ile ser  lhacilswobjthule thaveengured waind hirlouts areth ny makt till iatthouko delc npaelend lo ty tou ceeit waru hot  hesanytt mt the maslssfor fhe py myua
----- temperature: 0.75
----- Generating with seed: "saucy bark inferior far to hison your br"
saucy bark inferior far to hison your brafdssd elled thy sethoubrt sf are yis rginsue te mesemtr ce bom thhtcprulgith sy ans yirh stur dwine ooungi v ens meis mheseurhasis thile scodutit bouad ntlos inf ond with ring yhephy thigt cave t thetsenthoushoais n iil mewe meith ons val

In [18]:
gen_sonnet_(model)

----- temperature: 0.25
----- Generating with seed: "em yet so they mourn becoming of their w"
em yet so they mourn becoming of their with ear borngadd seef thet shill to urcele my doed thinfwhere in the fariress deefitper asupceoring shill of reevireand thees preivefto that ceiqunes butrs to hew thut shillh love whon and the eyes thine my seine thy nour fools of this saifors his and what whicth maise my groce no ded roting forsed in thy besmy forcemour guines whore no houly ang heftone and thou mayils make love lise arigut heave----- temperature: 0.75
----- Generating with seed: "em yet so they mourn becoming of their w"
em yet so they mourn becoming of their willand the braye therethile love an besundbethatkit all ald thot thou thougabst furse no can thou shaule deof chose this everys welo hisone meaneed wheir and mreathoot as the busts tery stor vargrouby thee my all this bricks wonts mise hight i theets thiewhin the faor dithet foo this my loves if the fordfaresedizeand do thines m

In [16]:
# from https://keras.io/examples/lstm_text_generation/

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    # print only 10th epoch
    if epoch%100==0:
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for temperature in [0.25, 0.75, 1.5]:
            print('----- temperature:', temperature)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()
    else: 
        pass

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [194]:
# fit model
model.fit(x, y, batch_size=32, epochs=100, verbose=2, callbacks=[print_callback])

Epoch 1/100
 - 27s - loss: 0.8847 - accuracy: 0.7301

----- Generating text after Epoch: 0
----- temperature: 0.25
----- Generating with seed: "son music to hear why hearst thou music "
son music to hear why hearst thou music and thee the world mouth re painted how a fomen forthing and so so thou the world i to keed for thy sunfeathert me dis caired thee beauty one womblong of my sweet were it not than my self a long have preasor of the fairt a a send the beauty on your self and thee thee belies summers with lost and contould me doth long hast where it not thou art i am not love and this paress by yet are gondless by t
----- temperature: 0.75
----- Generating with seed: "son music to hear why hearst thou music "
son music to hear why hearst thou music comed be endand the forghts on the ramethou akn that mull for all meame doth londs crouch pottricet it st oll whor be endle butl you tremes wore thoughtshenger muli shall the self for this pleing math not to be st leave whose be night dil

In [293]:
def gen_sonnet(model, seed):
    "generate sonnet from model given seed"

    for temperature in [0.25, 0.75, 1.5]:
        print('\n ==temperature:', temperature)

        generated = ''
        sentence = seed
        generated += sentence
        print('==Generating with seed: "' + sentence + '"')
        
        print('\n', seed)
        for line in range(13):
            
            sentence = sentence
            
            for i in range(400): # why 400?
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

            sonnet = sentence
            print('\n', sonnet)
            
    


In [ ]:
char_seed = "shall i compare thee to a summers day "

In [294]:
gen_sonnet(model, seed=char_seed)


 ==temperature: 0.25
==Generating with seed: "shall i compare thee to a summers day "

 shall i compare thee to a summers day 

 a e a ree eo beeo aiefaaafhrtta meo oe

 aaio o wth etitnoner ao esesooayt etre

 unyr earsuo a eay to jmosreeruooooooo 

 oeooedi ealhipa eyhtnoia e eye elo reo

 aoo o ooooo o oenieyieeayhtne elesomes

   eifoio uooeayhrg gteoo einesati wtst

 ysoo yeoo esy eaysabesaehomaehmti eres

  o ooeo ealllayeteelsyia iiaaa e eyhte

  a o eeyenhrle reay temi tnoitn eli o 

 eahsaeeoiayu oruoen vwnoh ui ety ineae

 esvsto oa o ooeaneneio  aaayriaye e oe

 piy cpiaaooooo o oefeey teteito eti tc

 eteeoooooooneaeaneehisoeaehiaehraid re

 ==temperature: 0.75
==Generating with seed: "shall i compare thee to a summers day "

 shall i compare thee to a summers day 

 ooo o oeoo o rrmgngse reo mua o oeoooo

 tia me reo ioo nellaiesrslbya eu rei a

  sltttlgvttuifooooo o eo enhme aaoeooa

 o oeeuftilayaaeanhreyhi ifelaaayhcia m

 sdo ouoooooo n npa meo ey elaefiae ino

 lrit

In [ ]:
# get stuck on vowel strings

In [ ]:
# run again more epochs 

In [300]:
# fit model
model_1000 = Sequential()
model_1000.add(LSTM(125, input_shape=(x.shape[1], x.shape[2])))
model_1000.add(Dense(len(chars), activation='softmax'))
model_1000.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_1000.fit(x, y, batch_size=32, epochs=1000, verbose=2, callbacks=[print_callback])

Epoch 1/1000
 - 28s - loss: 2.5487 - accuracy: 0.2603

----- Generating text after Epoch: 0
----- temperature: 0.25
----- Generating with seed: "d in quest to have extremea bliss in pro"
d in quest to have extremea bliss in proods of the pairted i an hing his sweet owfrrswere it not gildes my best is deep corredthen cruck a all this vairly gust in shallt ho heavens my beauty my stlean blackh prair threemplese cromjed in thy behtsi blown i not so muly formen in the cliand in thy cometitien the her loves wort so stapp tooukend gurron her ampeature weakness thy love no mersed fair and this thy prodst as tind me dowhrins ex
----- temperature: 0.75
----- Generating with seed: "d in quest to have extremea bliss in pro"
d in quest to have extremea bliss in proods of thee as thine on thremein to a ewerfllestion thou and me a parse be her loves ano withi ant not the world so plasoociou sit is fort yet keept art reasess yet not grow there freeloke thou belled so sweet mease woe should give thari

In [301]:
gen_sonnet(model_1000, seed=char_seed)


 ==temperature: 0.25
==Generating with seed: "shall i compare thee to a summers day "

 shall i compare thee to a summers day 

 eeeedee iii ouaiiii oa et eesasa  e et

 iooonaoo reo oeokseri  ate aiai a mka 

 e reoeeileee iteai o oaeesceeeeuii sa 

  a oueeeigueue aaei ceaa a elra  e esa

 oo o ooio ris otee ceaai amrto oeo elo

 peoeoetee o eraoiree cereteeoae o oooo

 ai ai a aie erfatea a elyei iey reyigd

 aaee eeoaiileei wnti ci ai a e mtcit r

 eeoo ei iileeihgisa aii o aei sa a ai 

 a e aaa ca e mrueeo eihy eirof oaeuaoo

 eetiilwcia a elyussrcaeaa aai  a fef t

 iliileei iti sa a ede ie reoeoetee o e

 o wrto ouoieo elofeeueeoae oai o roeo 

 ==temperature: 0.75
==Generating with seed: "shall i compare thee to a summers day "

 shall i compare thee to a summers day 

 aie etliiuo eoaai o aia e e ir telo ca

 ee iteataeo eihyusae reoo te mtooii e 

 ue aii y dsicme ii rilorrioiieeisrsai 

 isyilee  isae oeiooo oaai oa et ouilee

 teito uoiae ceaai lrae o ueueeoee reet

  e e

'lgi tiei thethrueelaei t aioookhriseem'

In [70]:
def gen_sonnet_(model):
    # Function invoked at end of each epoch. Prints generated text.
    

    start_index = random.randint(0, len(text) - maxlen - 1)
    for temperature in [0.25, 0.75, 1.5]:
        print('\n ----- temperature:', temperature)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"\n')

        entire_seq = generated
        for i in range(520):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char
          
            entire_seq = entire_seq + next_char
        
        
        for ind, char in enumerate(entire_seq):
            if ind%40==0:
                print(entire_seq[ind:ind+40])

   

In [71]:
gen_sonnet_(model)


 ----- temperature: 0.25
----- Generating with seed: "ll give thee memorythou by thy dials sha"

ll give thee memorythou by thy dials sha
dthup inkryeprtot whole dants with shefr
e mise ake should ifethendss thine ary s
heethou doth this fire thou wainut hewon
d thinost whet trall fubct noowon suelye
d for tha ghouland not this pyounds whit
bgencecryoun though but by thing ingrise
sthy fay age this boring by shuli lowh c
anturren my mare shay thou theee and far
tellone egeny whinedut shomlst in love h
and you he wool to peaniestind thy cante
ance for tise enfendso poindseefile make
 prideshes with thou porale wo chond thy
 comare or peetendtyald dispestreno faie

 ----- temperature: 0.75
----- Generating with seed: "ll give thee memorythou by thy dials sha"

ll give thee memorythou by thy dials sha
rlle thing home norceard thee a disbired
 in seidane no eyare of deefts life at e
vembrefinder of this sawnos bith ho shaj
gity loef and love your and the will no 
sunoth nei hewith oe chay

In [133]:
# def gen_sonnet_seed(model, start_ind, end_ind):
#     # Function invoked at end of each epoch. Prints generated text.
    

# #     start_index = random.randint(0, len(text) - maxlen - 1)
#     for temperature in [0.25, 0.75, 1.5]:
#         print('\n ----- temperature:', temperature)

#         generated = ''
# #         sentence = text[start_index: start_index + maxlen]
#         sentence = text[start_ind: end_ind]

# #         sentence = seed    
#         generated += sentence
        
#         print('----- Generating with seed: "' + sentence + '"\n')

#         entire_seq = generated
#         for i in range(520):
#             x_pred = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t, char_indices[char]] = 1.

#             preds = model.predict(x_pred, verbose=0)[0]
#             next_index = sample(preds, temperature)
#             next_char = indices_char[next_index]

#             sentence = sentence[1:] + next_char
          
#             entire_seq = entire_seq + next_char
        
        
#         for ind, char in enumerate(entire_seq):
#             if ind%40==0:
#                 print(entire_seq[ind:ind+40])


In [136]:
def gen_sonnet_seed(model, start_ind, end_ind):
    # Function invoked at end of each epoch. Prints generated text.
    

#     start_index = random.randint(0, len(text) - maxlen - 1)
    for temperature in [0.25, 0.75, 1.5]:
        print('\n ----- temperature:', temperature)

        generated = ''
        sentence = text[start_ind: end_ind]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"\n')

        entire_seq = generated
        for i in range(520):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char
          
            entire_seq = entire_seq + next_char
        
        
        for ind, char in enumerate(entire_seq):
            if ind%40==0:
                print(entire_seq[ind:ind+40])

   

In [108]:
sentence

'hings right true my heart and eyes have'

In [105]:
text[40:40+40]

'sethat thereby beautys rose might never '

In [83]:
char_seed = "shall i compare thee to a summers day "

In [94]:
char_seed = "hings right true my heart and eyes have"

In [93]:
gen_sonnet_(model)


 ----- temperature: 0.25
----- Generating with seed: "hings right true my heart and eyes have "

hings right true my heart and eyes have 
shy guiby be fors nowesald dearesing ere
whit i the firew should and noters besha
ve thesest conquitkand reedth thou ansta
y shars dighe hath wontherefor shouldsth
an he poon beauly of the prife do com ko
rnand nave when in this ik freest of lov
es love what nanpes should loot ous no u
nrinddofbit that groons illegr have your
 somowhind eies thou forsh reveeves thou
 suse thoues excarryon now yesw how do t
hey bithe so chish eaxy lysert his corer
 to taying berpenss wert my criebur my r
evero an thy mondors i dady ntimple thy 

 ----- temperature: 0.75
----- Generating with seed: "hings right true my heart and eyes have "

hings right true my heart and eyes have 
that i beind is wand roof on whise the f
ay and thine ofonoud yenter with sholl h
eathtns sweets bedone or that lives my b
eat farwerougar me dofornand kifto this 
norundss whese she broog 

In [96]:
char_seed

'hings right true my heart and eyes have'

In [110]:
gen_sonnet_seed(model, char_seed)


 ----- temperature: 0.25
----- Generating with seed: "hings right true my heart and eyes have"

hings right true my heart and eyes haveb
 o uiyon o yigtmiise isaduidsetrie hwet 
ienois ecibegiuaasistiu eptowntolkntorlt
etonu eseknos rin ee owempoes on iviiag 
sepiniiel o o o yar osecow o rot o a eis
hyn o o rtrensneessoe nufeweros i pakas 
hugtoyn resmsimyhe miese iknbiyhwefrs si
gatyugtsoo i a or al rsmig o i o eaeuyhe
 i a aleo e o rsetynoeusin on oket oehty
uwiu sadsoi atis ipesnossresetostbelefet
heetisno usigtemosisoeusoeteselsasestot 
ersutinurs heit i o i erwefo uetsoqeutdi
ythuhles o otreslsiter o i lvieys hsffie
finteo i i une al iaden o witsfinthses 

 ----- temperature: 0.75
----- Generating with seed: "hings right true my heart and eyes have"

hings right true my heart and eyes havee
trabe assinigwosos o otras ai utesi ah o
 o lvoyoylogtrosos erinosn tmehecul o te
tis epset sewi o rsehoeoeos o nertea o y
u eedo a aetiwiy o o o eebsesesoedoi nid
o o i o o utrlisnsbeti o rma

In [137]:
summer_start_ind = 9952
summer_end_ind = 9990

In [138]:
gen_sonnet_seed(model, summer_start_ind, summer_end_ind)


 ----- temperature: 0.25
----- Generating with seed: " shall i compare thee to a summers day"

 shall i compare thee to a summers daysa
esoe ibia ifisenie hwtesu ionftncaityhu 
oeso netrikes y ayosomiaesoeweisisbailns
liweouen emoteen o nreehlgita reneo inoe
oceofongs buiwsi aoreliaea lseeo ti inow
nste o e eseisoi o ioeldeaioohrotesnsrsa
 o ettmispgitk iflpoiatlse liuspsie esos
oeeooogeoo aoaeose o ennswte o esosa cie
 ltoe ili e etityo mti  odniseoe o erya 
esseeei uai uap eiliga asetoaybitfouaa e
n potittise o esdojesosoha oo ioeooo o r
eooresoecio aii ainesei soefisefi iti ci
oeceibid se o elsrmo ihe oe istiidtt eun
nwtteaosa lasesptsu otey oeenesi atret

 ----- temperature: 0.75
----- Generating with seed: " shall i compare thee to a summers day"

 shall i compare thee to a summers daye 
eewseheo o eliteto o riasoasye testi  lf
piseycoseif ifroeitifioo i etrmy gteei o
 eilfsoosisoyi o ttsllpo o yreoo o  e re
iso y e  esnisesosa ieteoolde ed eetresu
 erlwsergi aiinl a e ifoysditos

In [208]:
# # choose random index 
# start_index = random.randint(0, len(text) - maxlen - 1)
# # 40 character snippet to seed new sequence
# sentence = text[start_index: start_index + maxlen]

# # or 
# # sentence = char_seed

# # run through 400 characters
# for i in range(400):
#     # predictions 40 characters to choose by 61 possible characters
#     x_pred = np.zeros((1, maxlen, len(chars)))
#     for t, char in enumerate(sentence):
#         # fill in 1 at that character (row /40) in index of character (column)
#         x_pred[0, t, char_indices[char]] = 1.

#     # generate prob of next character (array of probabilites for each character)
#     preds = model.predict(x_pred, verbose=0)[0]
#     # pick character from distribution of pred (output=index)
#     next_index = sample(preds, temperature)
#     # use index to find character
#     next_char = indices_char[next_index]

#     # add character to sentence seeded, replace 1st character with last new character 
#     sentence = sentence[1:] + next_char

# #     sys.stdout.write(next_char)
# # sys.stdout.write(sentence)
# # sys.stdout.flush()
# print(sentence)

t they pay the vess chould not book ho l
